In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import pickle
import pywt
import os
import numpy as np
from PIL import Image, ImageTk
import re
import wikipedia

def w2d(img, mode='haar', level=1):
    imArray = img
    # Datatype conversions
    # Convert to grayscale
    imArray = cv2.cvtColor(imArray, cv2.COLOR_RGB2GRAY)
    # Convert to float
    imArray = np.float32(imArray)
    imArray /= 255
    # Compute coefficients
    coeffs = pywt.wavedec2(imArray, mode, level=level)

    # Process Coefficients
    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0

    # Reconstruction
    imArray_H = pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255
    imArray_H = np.uint8(imArray_H)

    return imArray_H

def scaling(img):
    color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)

    face = cv2.CascadeClassifier("C://Users//anith//OneDrive//Desktop//Classifiers//haarcascade_frontalface_default.xml")
    eye = cv2.CascadeClassifier("C://Users//anith//OneDrive//Desktop//Classifiers//haarcascade_eye.xml")

    face_val = face.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in face_val:
        gray_face = gray[y:y + h, x:x + w]
        color_face = color[y:y + h, x:x + w]
        eye_val = eye.detectMultiScale(gray_face)
        return color_face

def wavelet(img):
    scalled_raw_img = cv2.resize(img, (32, 32))
    img_har = w2d(img, 'db1', 5)
    scalled_img_har = cv2.resize(img_har, (32, 32))
    combined_img = np.vstack((scalled_raw_img.reshape(32 * 32 * 3, 1), scalled_img_har.reshape(32 * 32, 1)))
    return combined_img

# Function to open a file dialog and select an image file
def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        scaled_img = scaling(image)
        transformed_img = wavelet(scaled_img)
        value = best_clf1.predict(transformed_img.reshape(1, -1))
        val = value.item()
        key = [k for k, v in class_dict.items() if v == val]
        celebrity_name = key[0]
        celebrity = re.sub(r"_", " ", celebrity_name).title()
        result_label.config(text='The celebrity is: ' + celebrity)
        
        
        img = Image.open(file_path)
        img = img.resize((150, 250))
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk

cropped_path = "C://Users//anith//OneDrive//Desktop//Cropped"
celebrity_file_names_dict = {}

for img_dir in os.listdir(cropped_path):
    img_dir_path = os.path.join(cropped_path, img_dir)
    celebrity_name = img_dir.split('/')[-1]
    file_list = []
    for entry in os.scandir(img_dir_path):
        if entry.is_file():
            file_list.append(entry.path)
    celebrity_file_names_dict[celebrity_name] = file_list

class_dict = {}
count = 0
for celebrity_name in celebrity_file_names_dict.keys():
    class_dict[celebrity_name] = count
    count += 1
    
with open('CNN_best.pkl', 'rb') as file:
    best_clf1 = pickle.load(file)

root = tk.Tk()
root.title('Celebrity Recognition')
background_path = "C://Users//anith//Downloads//red-carpet-640x964.png"
canvas = tk.Canvas(root, width=400, height=640)
canvas.pack()
background_img = Image.open(background_path)
background_img = background_img.resize((400, 640))
background_tk = ImageTk.PhotoImage(background_img)
canvas.create_image(0, 0, anchor=tk.NW, image=background_tk)

root.geometry("400x640")

image_label = tk.Label(root)
image_label.place(x=120, y=50)
image_label.configure(borderwidth=0, highlightthickness=0)

browse_button = tk.Button(root, text='Upload Image', command=browse_file)
browse_button.place(x=160, y=350)

result_label = tk.Label(root, text='')
result_label.place(x=115,y=400)
result_label.configure(borderwidth=0, highlightthickness=0)

root.mainloop()